In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!git config --global credential.helper store

In [3]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [4]:
billsum = billsum.train_test_split(test_size=0.2)

In [5]:
billsum["train"][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 120500 of the Health and Safety Code is amended to read:\n120500.\nAs used in the Communicable Disease Prevention and Control Act, defined in Section 27, “sexually transmitted diseases” means diseases that are primarily transmitted through sexual contact.\nSEC. 2.\nSection 120505 of the Health and Safety Code is amended to read:\n120505.\nThe department shall develop and review plans and provide leadership and consultation for, and participate in, a program for the prevention and control of sexually transmitted diseases.\nSEC. 3.\nSection 120510 of the Health and Safety Code is amended to read:\n120510.\nThe department shall cooperate in the prevention, control, and cure of sexually transmitted diseases with all of the following:\n(a) Physicians and surgeons.\n(b) Medical schools.\n(c) Public and private hospitals, dispensaries, and clinics.\n(d) Administrators of public and private elementary

In [6]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [7]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [9]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [10]:
import evaluate

rouge = evaluate.load("rouge")

In [11]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [12]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir="finetune_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    use_ipex=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/intel/oneapi/intelpython/envs/pytorch-gpu/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: ''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly se

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.776521,0.122000,0.034300,0.101700,0.102000,19.000000
2,No log,2.534224,0.151000,0.062600,0.126500,0.126400,19.000000
3,No log,2.439753,0.176100,0.076700,0.146900,0.146900,19.000000
4,No log,2.392098,0.197700,0.091900,0.165900,0.165900,19.000000
5,No log,2.362346,0.198500,0.093200,0.165900,0.166000,19.000000
6,No log,2.342689,0.197700,0.093700,0.165900,0.165900,19.000000
7,No log,2.330341,0.197300,0.093500,0.165800,0.165800,19.000000
8,No log,2.322132,0.195900,0.092600,0.164400,0.164400,19.000000
9,2.585600,2.317513,0.196700,0.094000,0.164900,0.165000,19.000000
10,2.585600,2.315709,0.196600,0.094300,0.164900,0.165000,19.000000


/opt/intel/oneapi/intelpython/envs/pytorch-gpu/lib/python3.9/site-packages/intel_extension_for_pytorch/frontend.py:618: UserWarning: Linear BatchNorm folding failed during the optimize process.
  warnings.warn(
/opt/intel/oneapi/intelpython/envs/pytorch-gpu/lib/python3.9/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
2024-03-27 21:46:16,136 - absl - INFO - Using default tokenizer.
2024-03-27 21:46:40,137 - absl - INFO - Using default tokenizer.
2024-03-27 21:47:04,169 - absl - INFO - Using default tokenizer.
2024-03-27 21:47:28,219 - absl - INFO - Using default tokenizer.
2024-03-27 21:47:52,291 - absl - INFO - Using default tokenizer.
2024-03-27 21:48:16,338 - absl - INFO - Using default tokenizer.
2024-03-27 21:48:40,408 - absl - INFO - Using default tokenizer.
2024-03-27 21:49:

TrainOutput(global_step=620, training_loss=2.5335489580708166, metrics={'train_runtime': 251.8796, 'train_samples_per_second': 39.265, 'train_steps_per_second': 2.461, 'total_flos': 2677060833116160.0, 'train_loss': 2.5335489580708166, 'epoch': 10.0})

In [15]:
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

In [16]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from torch import no_grad

# Load your fine-tuned model and tokenizer
model_path = "finetune_model"  # Change this to your model's directory
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

def summarize(text):
    # Encode the text input
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")

    # Generate summary with your model
    with no_grad():
        summary_ids = model.generate(inputs['input_ids'], max_length=150, num_beams=4, early_stopping=True)

    # Decode the generated ids to a text string
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Example text
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."
# Get the summary
summary = summarize(text)
print(summary)


The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It will ask the ultra-wealthy and corporations to pay their fair share.


In [17]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import numpy as np
import time

model_checkpoint = "finetune_model"  # Change this to your model's directory
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Prepare your text
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."
inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")

# Ensure model is in evaluation mode
model.eval()

# Check if CUDA (GPU support) is available and use it, otherwise fall back to CPU

model = model.to("xpu")
inputs = inputs.to("xpu")

num_times = 100
warmup_rounds = 20

latency_list = []
for i in range(num_times):
    time_start = time.time()
    with torch.no_grad():
        outputs = model.generate(inputs['input_ids'], max_length=150, num_beams=4, early_stopping=True)
    if i >= warmup_rounds:
        latency_list.append(time.time() - time_start)

fp32_inference_time = np.mean(latency_list)
print(f"Inference time for FP32 took {fp32_inference_time:.3f} seconds")


Inference time for FP32 took 0.855 seconds


In [18]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import numpy as np
import time
import intel_extension_for_pytorch as ipex

model_checkpoint = "finetune_model"  # Change this to your model's directory
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Prepare your text
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."
inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")

# Ensure model is in evaluation mode
model.eval()

# Check if CUDA (GPU support) is available and use it, otherwise fall back to CPU
model = model.to("xpu")
inputs = inputs.to("xpu")

num_times = 100
warmup_rounds = 20

model = ipex.optimize(model, dtype=torch.bfloat16)
latency_list = []
with torch.no_grad():
    ########################### code changes ########################
    with torch.xpu.amp.autocast(enabled=True, dtype=torch.bfloat16):
    ########################### code changes ########################
        for i in range(num_times):
            time_start = time.time()
            outputs = model.generate(inputs['input_ids'], max_length=150, num_beams=4, early_stopping=True)
            if i >= warmup_rounds:
                latency_list.append(time.time() - time_start)

fp32_inference_time = np.mean(latency_list)
print(f"Inference time for FP32 took {fp32_inference_time:.3f} seconds")

/opt/intel/oneapi/intelpython/envs/pytorch-gpu/lib/python3.9/site-packages/intel_extension_for_pytorch/frontend.py:611: UserWarning: Conv BatchNorm folding failed during the optimize process.
  warnings.warn(
/opt/intel/oneapi/intelpython/envs/pytorch-gpu/lib/python3.9/site-packages/intel_extension_for_pytorch/frontend.py:618: UserWarning: Linear BatchNorm folding failed during the optimize process.
  warnings.warn(


Inference time for FP32 took 0.971 seconds
